In [ ]:
USER_FLAG = "--user"
!pip install {USER_FLAG} --upgrade "tfx[kfp]<2"
!pip install {USER_FLAG} apache-beam[interactive]

In [ ]:
import os
import IPython
if not os.getenv(""):
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tfx import v1 as tfx
import kfp
print('TensorFlow version:', tf.__version__)
print('TFX version: ',tfx.__version__)
print('KFP version: ',kfp.__version__)

In [ ]:
PROJECT_ID="vertex-my"
!gcloud config set project {PROJECT_ID}
BUCKET_NAME="tfx_pipeline_demo"
NAME_PIPELINE = "tfx-pipeline"
ROOT_PIPELINE = f'gs://{BUCKET_NAME}/root/{NAME_PIPELINE}'
MODULE_FOLDER = f'gs://{BUCKET_NAME}/module/{NAME_PIPELINE}'
OUTPUT_MODEL_DIR=f'gs://{BUCKET_NAME}/output_model/{NAME_PIPELINE}'
INPUT_DATA_DIR = 'gs://tfx_pipeline_input_data'

In [ ]:
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
#from tfx.components.statistics_gen.component import StatisticsGen
#from tfx.components.schema_gen.component import SchemaGen
#from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx_bsl.public import tfxio
from tfx.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
#from tfx.components.example_validator.component import ExampleValidator
#from tfx.components import Transform
from tensorflow_metadata.proto.v0 import schema_pb2
import os

from typing import List
#from absl import logging

In [ ]:
TRANSFORM_MODULE_PATH = 'file_transform.py'

In [ ]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft

NAMES = ['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']
LABEL = 'eyeDetection'

def preprocessing_fn(raw_inputs):
    processed_data = dict()
    for items in NAMES:
        processed_data[items]=raw_inputs[items]
    processed_data[LABEL] = raw_inputs[LABEL]

    return processed_data

In [ ]:
!gsutil cp file_transform.py {MODULE_FOLDER}/

In [ ]:
%%writefile trainer.py
from typing import List
from absl import logging
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2
COL_NAMES=['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']
LABEL="eyeDetection"
BATCH_SIZE_TRAIN = 40
BATCH_SIZE_EVAL = 20
def _input_fn(files,accessor,transform_output,size) -> tf.data.Dataset:
    dataset = accessor.tf_dataset_factory(
        files,
        tfxio.TensorFlowDatasetOptions(batch_size=size),
        schema=transform_output.raw_metadata.schema)
    tft_layer = transform_output.transform_features_layer()
    def apply_transform_fn(raw_input_features):
        features_transformed = tft_layer(raw_input_features)
        label_transformed = features_transformed.pop(LABEL)
        return features_transformed, label_transformed
    return dataset.map(apply_transform_fn).repeat()

def _get_serve_tf_examples_fn(model, transform_output):
    model.tft_layer = transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_sample):
        feature_spec_required = {
            k: v for k, v in transform_output.raw_feature_spec().items() if k != LABEL
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_sample,
            feature_spec_required)
        transformed_feat = model.tft_layer(parsed_features)
        return model(transformed_feat)
    return serve_tf_examples_fn

def _make_keras_model() -> tf.keras.Model:

    inputs_layer = [keras.layers.Input(shape=(1,), name=n) for n in COL_NAMES]
    lay = keras.layers.concatenate(inputs_layer)
    lay = keras.layers.Dense(16, activation='relu')(lay)
    lay = keras.layers.Dense(4, activation='relu')(lay)
    lay = keras.layers.Dense(16, activation='relu')(lay)
    lay = keras.layers.Dense(4, activation='relu')(lay)
    outputs_layer = keras.layers.Dense(1)(lay)

    model_classification = keras.Model(inputs=inputs_layer, outputs=outputs_layer)
    opti=keras.optimizers.Adam(1e-2)
    los=tf.keras.losses.BinaryCrossentropy(from_logits=False)
    metr=[keras.metrics.BinaryAccuracy()]
    model_classification.compile(
        optimizer=opti,loss=los,metrics=metr)
    return model_classification

def run_fn(fn_args: tfx.components.FnArgs):
    tf_transform = tft.TFTransformOutput(fn_args.transform_output)
    train_samples = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform,
        size=BATCH_SIZE_TRAIN)
    eval_samples = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform,
        size=BATCH_SIZE_EVAL)

    model_classification = _make_keras_model()
    model_classification.fit(
        train_samples,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_samples,
        validation_steps=fn_args.eval_steps)
    sign = {
        "serving_default": _get_serve_tf_examples_fn(model_classification, tf_transform),
    }
    model_classification.save(fn_args.serving_model_dir, save_format='tf',signatures=sign)


In [ ]:
!gsutil cp trainer.py {MODULE_FOLDER}/

In [ ]:
def _create_pipeline(pl_name, pipeline_root_folder, data_root,
                     module_file_transform, module_file_train, model_dir_save,
                     ) -> tfx.dsl.Pipeline:

    pass

In [ ]:
trainer_file="trainer.py"
file_transform=os.path.join(MODULE_FOLDER, TRANSFORM_MODULE_PATH)
file_train=os.path.join(MODULE_FOLDER, trainer_file)
pl_def_file = NAME_PIPELINE + '.json'

pl_runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=pl_def_file)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = pl_runner.run(
    _create_pipeline(
        pl_name=NAME_PIPELINE,
        pipeline_root_folder=ROOT_PIPELINE,
        data_root=INPUT_DATA_DIR,
        module_file_transform=file_transform,
        module_file_train=file_train,
        model_dir_save=OUTPUT_MODEL_DIR))